In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [10]:
pwd

'C:\\Users\\USER\\Documents\\Python Scripts'

In [2]:
def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction,feed_dict={xs:v_xs,keep_prob:1}) #keep prob as 1 这里的keep_prob是保留概率，即我们要保留的结果所占比例，它作为一个placeholder，在run时传入， 当keep_prob=1的时候，相当于100%保留，也就是dropout没有起作用
    correction_prediction = tf.equal(tf.arg_max(y_pre,1),tf.arg_max(v_ys,1)) #tf.equal(A, B)是对比这两个矩阵或者向量的相等的元素，如果是相等的那就返回True，反正返回False
    # mean( how many true predict = true data)
    accuracy = tf.reduce_mean(tf.cast(correction_prediction,tf.float32))
    result = sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys,keep_prob:1})
    return result

In [3]:
#define NN
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x,W):
    #stride [1,x_movement,y_movement,1]
    #stride[0] &[3] must =1
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    #stride [1,x_movement,y_movement,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [4]:
#define placeholder for input to NN
xs = tf.placeholder(tf.float32,[None,784])
ys = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs,[-1,28,28,1]) # [0]:-1 mean keep original shape  [3]: z_axis
#print(x_image.shape)  # [n_samples, 28,28,1]


In [5]:
## cov1 ##
W_conv1 = weight_variable([5,5,1,32]) #patch5*5 ,input_size=1,ouput_size=32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1) #wx+b output size 28x28x32
h_pool1 = max_pool_2x2(h_conv1)  # output size 14x14x32


In [6]:
##conv 2 ##
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2) #output size 14*14*64
h_pool2 = max_pool_2x2(h_conv2)  #output size 7*7*64

In [7]:
## fc1 layer ##
W_fc1 = weight_variable([7*7*64,1024]) # will flat the mattrx 7x7x64->7*7*64x1
b_fc1 = bias_variable([1024])
 #[n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1) # xW + b
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [8]:
## fc2 later 
W_fc2 = weight_variable([1024,10]) # input size 1024, output size 10
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

cross_entropy = tf.reduce_mean(
                    -tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))
train_step = tf.train.AdadeltaOptimizer(1e-4).minimize(cross_entropy)


sess= tf.Session()
init=tf.global_variables_initializer()
sess.run(init)

In [9]:
for i in range(500):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.3})
    if i % 50 == 0:
        print(compute_accuracy(
            mnist.test.images[:1000], mnist.test.labels[:1000]))

0.102
0.103
0.103
0.103
0.103
0.103


KeyboardInterrupt: 